In [50]:
import random
import numpy as np
import pandas as pd

from surprise import Reader, SVD
from surprise.model_selection import cross_validate
from surprise.prediction_algorithms.knns import KNNWithMeans
from surprise.model_selection import GridSearchCV


from utils.leave_one_last import leave_one_last_item



my_seed = 1203
random.seed(my_seed)
np.random.seed(my_seed)

path = '../data/movie_lens_small/ratings.csv'

movies = pd.read_csv(path)

df = movies.copy()
df.drop(columns=['timestamp'], inplace=True)
#las columnas deben tener estos nombres, y estar en este orden para que load_from_df las lea
df.columns = ["userID", "itemID", "rating"]

trainset, testset = leave_one_last_item(df, 'userID')

#creo un objeto de clase Reader para parsear la data del archivo
reader = Reader(line_format='user item rating',
                rating_scale=(np.min(df['rating']), np.max(df['rating'])))

train = Dataset.load_from_df(trainset[["userID", "itemID", "rating"]], reader)
test = Dataset.load_from_df(testset[["userID", "itemID", "rating"]], reader)


#instancio el modelo
# CF_itembased = KNNWithMeans(k=40, min_k=1, 
#                             sim_options={'name':'pearson',
#                                           'user_based':False,
#                                        },
#                             verbose=True)

# cv = cross_validate(CF_itembased, 
#                      train,
#                      measures = ['RMSE', 'MAE'],
#                      cv = 5,
#                      n_jobs = -1,
#                      )

param_grid = {
               "name": ["pearson"],
               "min_k": [1, 5, 12],
               "k": [20, 40, 50],
               "user_based": [False],
            }

gs = GridSearchCV(KNNWithMeans, param_grid, measures=["rmse", "mae"], cv=3)

gs.fit(train)

# best RMSE score
print(gs.best_score["rmse"])

# combination of parameters that gave the best RMSE score
print(gs.best_params["rmse"])


Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computi

# explicar porque usaria rmse o MAE para elegir el mejor modelo

In [53]:
CF_itembased = gs.best_estimator["rmse"]

CF_itembased.fit(train.build_full_trainset())

Computing the msd similarity matrix...
Done computing similarity matrix.


In [ ]:
#Como mido las metricas de test?
#puedo fitear a partir de otra metrica distinta a mae o rmse?


'''In the Surprise library, the "was_impossible" flag indicates that the model was unable to make a prediction for the given user-item pair. This can happen for two reasons:

    User or item is unknown: The user or item may not have been included in the training data. This can happen if the user or item is new to the system, or if it has been removed from the dataset.

    Insufficient data: The model may not have enough data to make a reliable prediction for the given user-item pair. This can happen if the user or item has not been rated by many other users, or if the ratings that are available are not very informative.

In either case, the "was_impossible" flag is a warning that the model's prediction should be treated with caution. If you are relying on the model to make accurate predictions, you may want to consider filtering out user-item pairs that are marked as "was_impossible".'''

- tengo que separar bien los datos de movielens, como dice el libro, por usuario, para que queden usuarios en test y train, de paso podria ver si puedo usar el dataset grande.

- Tambien, me falta entender que es "was imposible" que me devuelve el modelo

- Tengo que ver si puedo usar las metricas que vi, como Mean Average Precision o la que toma en cuenta el ranking

- Me falta ver como predecir un TOP N ranking

- Me tengo que crear yo una funcion o una clase que tenga un metodo, que reciba un dataframe, que tenga [USUARIO, ITEM, RATING], y que devuelva dos dataframe TRAIN y TEST, que intente separar por usuario en ambos dataframe.

- Voy a ver como manejo el asunto de que no tenga suficientes usuarios para que haya en train y test, que hago ahi??

- Cuantos usuarios para train y cuantos para test? por porcentaje?

- El paper hablaba de Leave One Last item, donde cada registro corresponde a (usuario, item) par por usuario. La ventaja es que maximiza el numero de transacciones en el dataset que pueden ser usadas para entrenar el recomendador. Para test en este metodo, solo la ultima transaccion del usuario es utilizada como test y la anteultima como validation. Los demas registros, se utilizan para train